In [1]:
import os 
os.chdir("/data/lab/proj_Q1")
print(os.getcwd())

/data/lab/proj_Q1


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType


In [3]:
# 创建SparkSession
spark = SparkSession.builder.config('spark.ui.port', 4055).appName("pyspark SQL basic example") .config("spark.sql.debug.maxToStringFields", "1000").getOrCreate()

(1) [2 points] Create two Spark dataframes ( df_http df_dns ) from the files
http.log.gz dns.log.gz in folders 00 to 05 (six folders in total). Convert the ts
column to Timestamp data type. Create two temp view named http_log and dns_log .

In [4]:
dns=spark.read.csv('dns_merged.csv',header=True)


In [5]:
http=spark.read.csv('http_merged.csv',header=True)


In [6]:
from pyspark.sql.functions import col, from_unixtime
http = http.withColumn("ts", from_unixtime(col("ts").cast("long")).cast("timestamp"))
dns=dns.withColumn("ts",from_unixtime(col("ts").cast("long")).cast("timestamp"))
http.createOrReplaceTempView('http')
dns.createOrReplaceTempView('dns')

In [7]:
# 示例查询
spark.sql("SELECT * FROM http").show(2)
spark.sql("SELECT * FROM dns").show(2)


+-------------------+------------------+--------------+---------+---------------+---------+-----------+------+---------------+--------------+-------+--------------------+----------------+-----------------+-----------+----------+----+----------+---------------+-------+----------+---------------+------+--------+--------+--------------+--------------+
|                 ts|               uid|     id.orig_h|id.orig_p|      id.resp_h|id.resp_p|trans_depth|method|           host|           uri|version|          user_agent|request_body_len|response_body_len|status_code|status_msg|tags|resp_fuids|resp_mime_types|proxied|orig_fuids|orig_mime_types|origin|referrer|username|orig_filenames|resp_filenames|
+-------------------+------------------+--------------+---------+---------------+---------+-----------+------+---------------+--------------+-------+--------------------+----------------+-----------------+-----------+----------+----+----------+---------------+-------+----------+---------------+---

(2) [2 points] With the http log data, filter the rows where the status code is 200 and
method is GET, sort in a descending order according to the accessed count of the uri .
Use Spark SQL API and Spark dataframe, seperately

In [8]:
Q2=spark.sql("""
select uri,count(*) as access_count from http
where status_code=200
and method='GET'
group by uri
order by access_count desc
""")
Q2.show()
Q2.createOrReplaceTempView('q2')

[Stage 4:==========================================>                (5 + 2) / 7]

+--------------------+------------+
|                 uri|access_count|
+--------------------+------------+
|                   /|        9473|
|/admin/config.php...|         556|
|  /main.php?logout=1|         194|
|/top.php?stuff=15...|         191|
|            /top.php|         179|
|/main.php?stuff=1...|         172|
|  /get_latest_id.php|         159|
|/admin/config.php...|         138|
|    /cacti/index.php|         129|
|/en-US/api/messag...|         118|
|          /index.php|         105|
|/phpmyadmin/index...|          77|
|             /cacti/|          68|
|        /phpmyadmin/|          56|
|        /favicon.ico|          55|
|              /admin|          42|
|  /scripts/index.php|          40|
|             /icons/|          39|
|/en-US/api/search...|          39|
|  /cgi-bin/index.php|          37|
+--------------------+------------+
only showing top 20 rows



In [9]:
from pyspark.sql.functions import col, count, desc

# 核心逻辑
result = (
    http
    .filter((col("status_code") == 200) & (col("method") == "GET"))  
    .groupBy("uri")                                                
    .agg(count("*").alias("access_count"))                          
    .orderBy(desc("access_count"))                                 
)

result.show()  # 显示结果

[Stage 7:==========================================>                (5 + 2) / 7]

+--------------------+------------+
|                 uri|access_count|
+--------------------+------------+
|                   /|        9473|
|/admin/config.php...|         556|
|  /main.php?logout=1|         194|
|/top.php?stuff=15...|         191|
|            /top.php|         179|
|/main.php?stuff=1...|         172|
|  /get_latest_id.php|         159|
|/admin/config.php...|         138|
|    /cacti/index.php|         129|
|/en-US/api/messag...|         118|
|          /index.php|         105|
|/phpmyadmin/index...|          77|
|             /cacti/|          68|
|        /phpmyadmin/|          56|
|        /favicon.ico|          55|
|              /admin|          42|
|  /scripts/index.php|          40|
|             /icons/|          39|
|/en-US/api/search...|          39|
|  /cgi-bin/index.php|          37|
+--------------------+------------+
only showing top 20 rows



(3) [2 points] Use Spark SQL to join the http_log and dns_log tables by uid , and
calculate the percentage of proto=tcp for each uri group found in taks (2).


In [10]:
spark.sql("""
WITH joined_table AS (
    SELECT 
        http.uid,
        http.uri,  
        dns.proto
    FROM http
    INNER JOIN dns
        ON http.uid = dns.uid
    WHERE http.uri IN (SELECT uri FROM q2)
)
SELECT 
    uri,
    100.0 * COUNT_IF(proto = 'tcp') / COUNT(*) AS tcp_percent
FROM joined_table
GROUP BY uri
""").show()

+---+-----------+
|uri|tcp_percent|
+---+-----------+
+---+-----------+

